# Deploy inference Server using KFServing

In [1]:
from kubernetes import client

In [2]:
from kfserving import KFServingClient, constants, utils, V1alpha2EndpointSpec, V1alpha2PredictorSpec, V1alpha2SKLearnSpec, V1alpha2InferenceService, V1alpha2InferenceServiceSpec
from kubernetes.client import V1ResourceRequirements, V1ObjectMeta

In [3]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION
kind = constants.KFSERVING_KIND
namespace = 'explainable-ml-engine'
storage_uri = 'gs://deeploy-ml/explainable-ml-engine/model'
service_account_name = 'kfserving-service-credentials'

In [ ]:
KFServing = KFServingClient()
# https://cloud.google.com/docs/authentication/getting-started
KFServing.set_credentials(storage_type='GCS', 
                          namespace=namespace,
                          credentials_file='tmp/gcp.json',
                          service_account=service_account_name)

In [ ]:
default_endpoint_spec = V1alpha2EndpointSpec(
    predictor=V1alpha2PredictorSpec(
        service_account_name=service_account_name,
        sklearn=V1alpha2SKLearnSpec(
            storage_uri=storage_uri,
            runtime_version='v0.4.1',
            resources=V1ResourceRequirements(
                requests={'cpu':'100m','memory':'256Mi'},
                limits={'cpu':'250m', 'memory':'1Gi'}
                )
            )
        )
    )

In [ ]:
inference_svc = V1alpha2InferenceService(
    api_version=api_version,
    kind=kind,
    metadata=V1ObjectMeta(
        name='iris-test', 
        namespace=namespace
    ),
    spec=V1alpha2InferenceServiceSpec(
        default=default_endpoint_spec
    )
)

In [ ]:
KFServing.create(inference_svc)

In [ ]:
KFServing.delete('iris-test', namespace=namespace)

In [ ]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))